In [2]:
!pip install chromedriver_autoinstaller

In [3]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
from IPython.display import clear_output
import chromedriver_autoinstaller
from selenium import webdriver
from tqdm import tqdm

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [4]:
def get_page(pagenum) :
    page_base = 'https://finance.naver.com/research/industry_list.nhn?keyword=&brokerCode=&writeFromDate=&writeToDate=&searchType=upjong&upjong=%C5%EB%BD%C5&x=57&y=22&page={}'
    page = page_base.format(pagenum)
    return page

In [5]:
def get_links(page) :
    html = requests.get(page).text
    time.sleep(0.1)
    soup = BeautifulSoup(html, 'html.parser', )
    
    links_all = soup.select('div.box_type_m tr > td > a')
    links = []
    for lll in links_all:
        if 'industry_read.nhn?' in lll['href'] :
            link = 'https://finance.naver.com/research/' + lll['href']
            links.append(link)
            
    return links

In [10]:
def get_report(link) :
    html = requests.get(link).text
    time.sleep(0.1)
    soup = BeautifulSoup(html, 'html.parser')
    report = []
#     stock = soup.select('em')[0].text
#     target = soup.select('em > strong')[0].text
#     reco = soup.select('em.coment')[0].text
    
    title_del = soup.select('th.view_sbj > p.source')[0].text
    title = soup.select('th.view_sbj')[0].text.replace(title_del,'').strip()
    repor = soup.select('div.box_type_m.box_type_m3 > table > tr > td.view_cnt > div > p')
#     report = soup.select('td.view_cnt > div')[0].text.replace('\n','').strip()
    
    company = title_del.split('|')[0]
    date = title_del.split('|')[1]
#     pdf = soup.select('a.con_link')[0]['href']
    
    for i in range(len(repor)):
        report.append(repor[i].text.strip())
    
    if repor == [] :
        repor = soup.select('div.box_type_m.box_type_m3 > table > tr > td.view_cnt > div')
        report.append(repor[0].text.strip())
#     pdf = soup.select('a.con_link')[0]['href']
    
#     result = [stock, target, reco, title, report, company, date, pdf]
    result = [title, report, company, date]
    return result

In [7]:
def get_stock_report(totalpage, fname) :
    
    links = []
    for i in tqdm_notebook(range(1,totalpage),desc = '점검중') :
        page = get_page(i)
        links += get_links(page)
        
    print('{}개의 산업분석 리포트를 찾았습니다.'.format(len(links)))
    print('리포트 수집을 진행합니다.\n')
    
    results = []
    for link in tqdm_notebook(links, desc ='리포트') :
        result = get_report(link)
        results.append(result)
        
    datas = pd.DataFrame(results)
#     datas.columns = ['종목명','목표가','의견','제목','내용','회사','일자','보고서']
    datas.columns = ['제목','내용','회사','일자']
    datas.to_csv(fname)
    print('{}개의 산업분석 리포트 수집 완료.'.format(len(results)))

In [8]:
totalpage = 13
fname = '산업분석리포트.csv'

In [9]:
get_stock_report(totalpage,fname)

<ipython-input-7-2beb5773488b>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1,totalpage),desc = '점검중') :


점검중:   0%|          | 0/12 [00:00<?, ?it/s]

360개의 산업분석 리포트를 찾았습니다.
리포트 수집을 진행합니다.



<ipython-input-7-2beb5773488b>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for link in tqdm_notebook(links, desc ='리포트') :


리포트:   0%|          | 0/360 [00:00<?, ?it/s]

360개의 산업분석 리포트 수집 완료.
